### Add model, connection and device to settings:

In [1]:
# библиотеки
import os
import sys
import inspect
import numpy as np
import re
import math
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from hybriddomain.envs.hs.model.model_main import ModelNet as Model
from hybriddomain.settings.settings_main import Settings
from ddeint import ddeint

# пути и названия файлов
modelFileName = 'problems/hutchinson_x_equation'
conn_name = "connection.json"
device_conf_name = "devices.json"

#Ввод пароля для инициализации пользователя
model = Model()
model.io.loadFromFile(modelFileName)
settings = Settings(model, conn_name, device_conf_name, conn_name)

INFO:settings_main:device_conf_rpath
INFO:settings_main:devices.json


Please enter password for user imaslenikov:


 ·········


In [2]:
print("equations:")
print(model.equations)
print("params:")
print(model.params)
print("paramValues:")
print(model.paramValues)

equations:
[["U'= a*x*(1-U(t-1)/b)*U"]]
params:
['a', 'b']
paramValues:
[{'a': 2, 'b': 1}, {'a': 3, 'b': 1}]


### Run solver:

In [3]:
# запуск процесса решения уравнения
from hybriddomain.solvers.hs.remoterun.progresses.progress_notebook import ProgressNotebook
from hybriddomain.solvers.hs.postproc.results.results_main import ResultPostprocNet as ResultPostproc
progress = ProgressNotebook(100, prefix='solving')

progress_params = ProgressNotebook(len(model.paramValues), prefix='param')
# display(progress_params.progress)

progress_solving = ProgressNotebook(100, prefix='solving')
# display(progress_solving.progress)
# progress_solving.succ(0)

result_postproc = ResultPostproc(modelFileName)

from hybriddomain.solvers.hs.remoterun.remoterun import remoteProjectRun

clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))

# parametrisation:
def run(event):
    model = Model()
    model.io.loadFromFile(modelFileName)
    
    for params_idx in range(len(model.paramValues)):
        
        progress_params.succ(params_idx)
        progress_solving.set_prefix("solving")
        
        # changing param:
        model.defaultParamsIndex = params_idx
        model.io.saveToFile(settings.paths["hd"]["json"])
        model = Model()
        model.io.loadFromFile(modelFileName)
        
        remoteProjectRun(settings, 1, progress_solving,remove_old=False)
        
        # result_postproc.extract_out(params_idx, results_params)
        result_postproc.rename_out(params_idx)
    display(Javascript("alert('Done');"))
    
bTest.on_click(run)

display(progress_params.progress)
display(progress_solving.progress)

#remoteProjectRun(settings, 1, progress)
display(bTest)

IntProgress(value=0, description='param: ', max=1)

IntProgress(value=0, description='solving: ', max=99)

Button(description='run hutchinson_x_equation ', style=ButtonStyle())

### Results:

In [4]:
model.readResults(result_format=1)
# available result files:
print("result files: ", model.results_paths)
# available plot files:
print("plot files: ", model.plots_paths)

result files:  {'U': ['problems/hutchinson_x_equation/out/U_seq0.out', 'problems/hutchinson_x_equation/out/U_seq1.out']}
plot files:  {'U': ['problems/hutchinson_x_equation/out/U_seq0.mp4', 'problems/hutchinson_x_equation/out/U_seq1.mp4']}


In [5]:
# объекте model хранится вся информация по решению задачи
# get results (for TRAC-11 result_format=1):
model.readResults(result_format=1)
model.results_paths
# available results:
print(model.results_arrays.keys())

dict_keys(['U'])


### Video:

In [6]:
# available plots:
model.plots_paths

{'U': ['problems/hutchinson_x_equation/out/U_seq0.mp4',
  'problems/hutchinson_x_equation/out/U_seq1.mp4']}

In [7]:
display(HTML(model.get_video('U')))
display(HTML(model.get_video('U',1)))

### U(x, t-fixed),V(x, t-fixed):

In [10]:
# отрисовывает результат
%matplotlib inline
time = model.results_arrays['U']['timevalues']

times = model.results_arrays['U']['timevalues']

result = model.results_arrays['U']['resvalues']
print(len(result))
def f(t,x):
    result_x = np.array([result[time][x][t] for time in times])
    plt.plot(time, result_x, lw=2)
w = interactive(f,t=(0,len(result[0][0])-1,1),x=(0,len(result[0])-1,1))
display(w)

302


interactive(children=(IntSlider(value=50, description='t'), IntSlider(value=0, description='x', max=1), Output…